# 3. Data preprocessing
 3.1 Data cleansing
 3.2 Data reduction
 3.3 Data transformation
 3.4 Data enrichment
 3.5 Data validation

#### Import modules and data

In [ ]:
%config Completer.use_jedi = False

from sklearn.preprocessing import StandardScaler    # Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.decomposition import FactorAnalysis
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing, svm
import statistics
import statsmodels.api as sm
import plotly.express as px    # Creates entire figures
import seaborn as sns   # Data visuals
import matplotlib.pyplot as plt  # Plots
import numpy as np      # Working with arrays and math
import pandas as pd     # For data analiysis and data process
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# Read the two datasets into DataFrames
df_hadcrut = pd.read_csv('./datasets/hadcrut-surface-temperature-anomaly.csv')
df_co2 = pd.read_csv('./datasets/owid-co2-data.csv')
print("Hadcrut-data consists of:", df_hadcrut.shape[0],"rows")
print("Owid-co2-data consists of:", df_co2.shape[0],"rows")

#### 3.1 Cleaning Process: HADCRUT-SURFACE-TEMPERATURE-ANOMALY

In [ ]:
## Rename columns
df_hadcrut = df_hadcrut.rename(columns=str.lower)
df_hadcrut = df_hadcrut.rename(columns={'entity': 'country'})
df_hadcrut = df_hadcrut.rename(columns={'code': 'iso_code'})
## Check NaN-rate for columns
def nan_rate (selected_column):
    nan_rate = df_hadcrut[selected_column].isna().mean()
    print(f"NaN-Rate der Spalte '{selected_column}': {nan_rate:.2%}")
nan_rate ('country')
nan_rate ('iso_code')
nan_rate ('year')
nan_rate ('surface temperature anomaly')
#print(df_hadcrut.isnull().sum())
df_hadcrut.loc[df_hadcrut['country'] == 'Micronesia', 'iso_code'] = 'FSM'
## If a NaN value or duplicate is found, delete the line
df_hadcrut.dropna(inplace=True)
df_hadcrut.drop_duplicates(inplace=True)
row1 = df_hadcrut.shape[0]
## Identify outliers with a boxplot
plt.boxplot(df_hadcrut['surface temperature anomaly'])
plt.ylabel('Surface Temperature Anomaly')
plt.xlabel('Boxplot')
plt.title('Identify outliers ')
plt.show()
# Delete outliners with the Z-score method:
z_scores = np.abs((df_hadcrut['surface temperature anomaly'] - df_hadcrut['surface temperature anomaly'].mean()) / df_hadcrut['surface temperature anomaly'].std())
# Define a limit above which a value is considered an outlier (Z-score > 3)
threshold = 3
# Entferne die Zeilen, die den Schwellenwert überschreiten
df_hadcrut = df_hadcrut[z_scores <= threshold]
row2 = df_hadcrut.shape[0]
print("Deleted rows:", row1 - row2)
print("Hadcrut-data consists of:", df_hadcrut.shape[0],"rows")

#### 3.1 Cleaning Process: OWDI-CO2-DATA

In [ ]:
# If a NaN value or duplicate is found, delete the line
row1 = df_co2.shape[0]
df_co2.dropna(inplace=True)
df_co2.drop_duplicates(inplace=True)
# Delete from the 'country' column all substrings which are in brackets
df_co2['country'] = df_co2['country'].str.replace(r'\s*\(.*\)', '')
row2 = df_co2.shape[0]
print("Deleted rows:", row1 - row2)
print("Owid-co2-data consists of:", df_co2.shape[0],"rows")

#### Merge Owid-co2 and Hadcrut

In [ ]:
# Merge the two DataFrames on the country and year and iso_code columns
# the 'iso_code' field is important so that no 'non-countries' are selected from owid-co2
df = df_hadcrut.merge(df_co2, on=['country', 'iso_code', 'year'])
# Identify the target variable, which is the anomaly temperature
target = 'surface temperature anomaly'
# Select a maximum of 15 columns from the merged DataFrame, including the target variable
selected_columns = ['country', 'iso_code', 'year'] + [target] + list(df.filter(like='co2')[:15]) + list(df.filter(like='ghg')[:1]) + ['nitrous_oxide', 'methane', 'gdp', 'population']
# Create a new DataFrame with the selected columns
df_pre_processed = df[selected_columns]
# Delete unnecessary columns
to_drop = ['cement_co2_per_capita',
           'co2_including_luc',
           'co2_including_luc_growth_abs',
           'co2_including_luc_growth_prct',
           'co2_including_luc_per_capita',
           'co2_including_luc_per_gdp',
           'share_global_cement_co2',
           'share_global_co2',
           'share_global_co2_including_luc',
           'share_global_coal_co2',
           'share_global_cumulative_cement_co2',
           'share_global_cumulative_co2',
           'share_global_cumulative_co2_including_luc',
           'share_global_cumulative_coal_co2',
           'share_global_cumulative_gas_co2',
           'share_global_cumulative_luc_co2',
           'share_global_cumulative_oil_co2',
           'share_global_cumulative_other_co2',
           'share_global_flaring_co2',
           'share_global_gas_co2',
           'share_global_luc_co2',
           'share_global_oil_co2',
           'share_global_other_co2',
           'co2_including_luc_per_unit_energy',
           'co2_per_capita',
           'co2_per_gdp',
           'co2_per_unit_energy',
           'coal_co2_per_capita',
           'consumption_co2_per_capita',
           'consumption_co2_per_gdp',
           'cumulative_cement_co2',
           'cumulative_co2_including_luc',
           'cumulative_coal_co2',
           'cumulative_flaring_co2',
           'cumulative_gas_co2',
           'cumulative_luc_co2',
           'cumulative_oil_co2',
           'cumulative_other_co2',
           'flaring_co2_per_capita',
           'gas_co2_per_capita',
           'land_use_change_co2_per_capita',
           'oil_co2_per_capita',
           'other_co2_per_capita',
           'other_industry_co2',
           'share_global_cumulative_flaring_co2',
           'temperature_change_from_co2',
           'trade_co2_share',
           'ghg_excluding_lucf_per_capita',
           'ghg_per_capita',
           'share_of_temperature_change_from_ghg',
           'iso_code',
           'total_ghg',
           'total_ghg_excluding_lucf',
           'temperature_change_from_ghg',
           'cement_co2',
           'co2_growth_abs',
           'co2_growth_prct',
           'consumption_co2',
           'cumulative_co2',
           'land_use_change_co2',
           'trade_co2']
df_pre_processed.drop(to_drop, inplace=True, axis=1)
# Sort df_analyse
desired_order = ['country', 'year', 'gdp', 'population', 'co2', 'coal_co2', 'flaring_co2',
                 'gas_co2', 'methane', 'nitrous_oxide', 'oil_co2', 'surface temperature anomaly']
df_pre_processed = df_pre_processed[desired_order]
# Rename 'surface temperature anomaly' to sta
df_pre_processed.rename(
    columns={'surface temperature anomaly': 'sta'}, inplace=True)
# Check for duplicates
df_pre_processed.drop_duplicates()
# Convert the object column 'country' to a numeric column 'country_id'
df_pre_processed['country_id'] = pd.factorize(df_pre_processed['country'])[0]
# Create a new dataframe 'df_country', which contains the assignment 'country_id' and 'country'
df_country = df_pre_processed[['country', 'country_id']].copy()
df_country.drop_duplicates(inplace=True)
# Move the column 'country_id' to the first position
column_order = ['country_id'] + \
    [col for col in df_pre_processed.columns if col != 'country_id']
df_pre_processed = df_pre_processed.reindex(columns=column_order)
# Set country_id and year as index
#df_pre_processed.set_index(['country_id', 'year'], inplace=True)
# Delete column 'country' so that the dataframe contains only numeric values
df_pre_processed.drop('country', axis=1, inplace=True)
# Save the new DataFrame as a CSV file
df_pre_processed.to_csv('./datasets/datas_pre_processed.csv')

In [ ]:
'''target_variable = df['sta']
features = df.drop('sta', axis=1)

# Min-Max-Normalisierung der Features
normalized_features = (features - features.min()) / (features.max() - features.min())
'''